# SmoothQuant Group Size Analysis

In this notebook, we analyze the impact of different group sizes on model perplexity when using group quantization in SmoothQuant.

In [30]:
!git clone https://github.com/adithyab100/smoothquant-mixedprecision.git
%cd smoothquant-mixedprecision
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install transformers==4.36.0 accelerate datasets zstandard
!python setup.py install

fatal: destination path 'smoothquant-mixedprecision' already exists and is not an empty directory.
/Users/ningshanma/Desktop/fa24/6.5940/smoothquant-mixedprecision/examples/smoothquant-mixedprecision/smoothquant-mixedprecision
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
ERROR: Could not find a version that satisfies the requirement torch==1.12.1+cu113 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==1.12.1+cu113
running install
/Users/ningshanma/miniconda3/envs/caveman/lib/python3.12/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021

In [32]:
%reload_ext autoreload
%autoreload 2

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
from transformers.models.llama.modeling_llama import (
    LlamaAttention,
    LlamaDecoderLayer,
    LlamaForCausalLM,
    LlamaMLP,
)
from transformers import LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import quantize_llama_like, quantize_opt
import tqdm
import gc
from functools import partial

ModuleNotFoundError: No module named 'smoothquant.smooth'

In [ ]:
%reload_ext autoreload
%autoreload 2

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
from transformers.models.llama.modeling_llama import (
    LlamaAttention,
    LlamaDecoderLayer,
    LlamaForCausalLM,
    LlamaMLP,
)
from transformers import LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import quantize_llama_like, quantize_opt
import tqdm
import gc
from functools import partial

ModuleNotFoundError: No module named 'smoothquant.smooth'

In [ ]:
%reload_ext autoreload
%autoreload 2

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
from transformers.models.llama.modeling_llama import (
    LlamaAttention,
    LlamaDecoderLayer,
    LlamaForCausalLM,
    LlamaMLP,
)
from transformers import LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import quantize_llama_like, quantize_opt
import tqdm
import gc
from functools import partial

ModuleNotFoundError: No module named 'smoothquant.smooth'

In [ ]:
%reload_ext autoreload
%autoreload 2

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
from transformers.models.llama.modeling_llama import (
    LlamaAttention,
    LlamaDecoderLayer,
    LlamaForCausalLM,
    LlamaMLP,
)
from transformers import LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import quantize_llama_like, quantize_opt
import tqdm
import gc
from functools import partial

ModuleNotFoundError: No module named 'smoothquant.smooth'

In [ ]:
%reload_ext autoreload
%autoreload 2

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
from transformers.models.llama.modeling_llama import (
    LlamaAttention,
    LlamaDecoderLayer,
    LlamaForCausalLM,
    LlamaMLP,
)
from transformers import LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import quantize_llama_like, quantize_opt
import tqdm
import gc
from functools import partial

ModuleNotFoundError: No module named 'smoothquant.smooth'

In [ ]:
%reload_ext autoreload
%autoreload 2

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
from transformers.models.llama.modeling_llama import (
    LlamaAttention,
    LlamaDecoderLayer,
    LlamaForCausalLM,
    LlamaMLP,
)
from transformers import LlamaTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import quantize_llama_like, quantize_opt
import tqdm
import gc
from functools import partial

ModuleNotFoundError: No module named 'smoothquant.smooth'

In [ ]:
class Evaluator:
    def __init__(self, dataset, tokenizer, device, n_samples=10):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        self.dataset = tokenizer(
            "\n\n".join(dataset["text"]), return_tensors="pt"
        ).input_ids.to(device)

        self.n_samples = n_samples

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        nlls = []
        n_samples = self.n_samples if self.n_samples else self.dataset.size(1) // 2048
        for i in tqdm.tqdm(range(n_samples), desc="Evaluating..."):
            batch = self.dataset[:, (i * 2048) : ((i + 1) * 2048)].to(model.device)
            with torch.no_grad():
                lm_logits = model(batch).logits
            shift_logits = lm_logits[:, :-1, :].contiguous().float()
            shift_labels = self.dataset[:, (i * 2048) : ((i + 1) * 2048)][:, 1:]
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1)
            )
            neg_log_likelihood = loss.float() * 2048
            nlls.append(neg_log_likelihood)

        return torch.exp(torch.stack(nlls).sum() / (n_samples * 2048))

In [ ]:
def evaluate_group_size(model_path, group_sizes, device="cuda" if torch.cuda.is_available() else "cpu"):
    # Load tokenizer and dataset
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
    evaluator = Evaluator(dataset, tokenizer, device, n_samples=10)  # Reduced samples for faster testing
    
    perplexities = []
    
    for group_size in group_sizes:
        print(f"\nTesting group size: {group_size}")
        
        # Load model fresh each time to avoid contamination
        model = AutoModelForCausalLM.from_pretrained(
            model_path, 
            torch_dtype=torch.float32 if device == "cpu" else torch.float16,
            device_map=device
        )
        
        # Quantize model with current group size
        model = quantize_model(
            model,
            weight_quant="per_group",
            act_quant="per_token",
            quantize_bmm_input=True,
            group_size=group_size,
            input_feat=None  # No input features available
        )
        
        # Evaluate
        ppl = evaluator.evaluate(model)
        perplexities.append(ppl)
        print(f"Perplexity for group size {group_size}: {ppl}")
        
        # Clear memory
        del model
        torch.cuda.empty_cache()
    
    return perplexities

In [ ]:
def plot_results(group_sizes, perplexities, output_path="group_size_perplexity.png"):
    plt.figure(figsize=(10, 6))
    plt.plot(group_sizes, perplexities, 'bo-')
    plt.xlabel('Group Size')
    plt.ylabel('Perplexity')
    plt.title('Model Perplexity vs. Group Size')
    plt.grid(True)
    plt.yscale('log')  # Log scale for perplexity
    plt.xscale('log')  # Log scale for group size
    plt.savefig(output_path)
    plt.close()

In [ ]:
# Configuration
model_path = "facebook/opt-125m"  # Using OPT-125M which is publicly available
group_sizes = [8, 16, 32, 64, 128]  # Different group sizes to test

# Run evaluation
perplexities = evaluate_group_size(model_path, group_sizes)

# Plot results
plot_results(group_sizes, perplexities)

# Print final results
print("\nFinal Results:")
for size, ppl in zip(group_sizes, perplexities):
    print(f"Group Size: {size}, Perplexity: {ppl}")